In [ ]:
from glob import glob
import pandas as pd
import os
from pathlib import Path
from PipeLine import *

In [ ]:
from dask.distributed import Client
from dask import delayed
from dask import compute
client = Client(n_workers=4)
client

In [ ]:
database = '/Users/amirhsi_mini/trjs_extractions/'
all_groups = glob(database+"N*_all/")
geometry = 'cylindrical'

In [ ]:
all_properties_file = '/Users/amirhsi_mini/trjs_analysis_bug/properties-all_in_one.csv'
all_properties = pd.read_csv(all_properties_file,index_col=0)

In [ ]:
# use this if there is no "properties-all_in_one.csv" file
csv_files = glob("./N*-extraction_all/*all*.csv")
properties_csvs = glob("./N*-analyze_bug/N*-properties.csv")

all_properties = []
for properties_csv in properties_csvs:
    df = pd.read_csv(properties_csv,index_col=0)
    all_properties.append(df) 
all_properties = pd.concat(all_properties)
all_properties.reset_index(inplace=True,drop=True)

In [ ]:
def analyze_segments(simulations_segments_files, geomertry, save_to):
    rhist_ensembles_crd = PipeLine.simulation_from_segments(simulations_segments_files,'rHistsCrd', geometry,['_rHistsCrd.csv','_rEdgesCrd.csv'], save_to=save_to)
    rhist_ensembles_mon = PipeLine.simulation_from_segments(simulations_segments_files,'rHistsMon', geometry,['_rHistsMon.csv','_rEdgesMon.csv'], save_to=save_to)
    _, _ = PipeLine.distributions_generator(rhist_ensembles_crd, all_properties, 'dcrowd',geometry,'radial','Crd',normalized=False,segments=True, save_to=save_to)
    _, _ = PipeLine.distributions_generator(rhist_ensembles_mon, all_properties, 'dmon', geometry,'radial','Mon',normalized=False,segments=True, save_to=save_to)

    zhist_ensembles_crd = PipeLine.simulation_from_segments(simulations_segments_files,'zHistsCrd',geometry,['_zHistsCrd.csv','_zEdgesCrd.csv'], save_to=save_to)
    zhist_ensembles_mon = PipeLine.simulation_from_segments(simulations_segments_files,'zHistsMon',geometry,['_zHistsMon.csv','_zEdgesMon.csv'], save_to=save_to)
    _, _ = PipeLine.distributions_generator(zhist_ensembles_crd, all_properties, 'dcrowd', geometry,'longitudinal','Crd',normalized=False,segments=True, save_to=save_to)
    _, _ = PipeLine.distributions_generator(zhist_ensembles_mon, all_properties, 'dmon', geometry,'longitudinal','Mon',normalized=False,segments=True, save_to=save_to)

    thetahist_ensembles_crd = PipeLine.simulation_from_segments(simulations_segments_files,'thetaHistsCrd', geometry,['_thetaHistsCrd.csv','_thetaEdgesCrd.csv'], save_to=save_to)
    thetahist_ensembles_mon = PipeLine.simulation_from_segments(simulations_segments_files,'thetaHistsMon', geometry,['_thetaHistsMon.csv','_thetaEdgesMon.csv'], save_to=save_to)

In [ ]:
def whole_all_group_analyze(group_files_path, parent_dir, simulation_type, geometry):
    sim_files = glob(group_files_path+"/N*/N*.csv")
    parent_path = group_files_path.split('trjs_extraction')[0]+"/"+parent_dir+"/"
    group_name = group_files_path.split("/")[-2].split('-')[0]+'-'+simulation_type+'-analysis-whole_simulations'
    group_path = parent_path+group_name
    Path(group_path).mkdir(parents=True, exist_ok=True)
    save_to = group_path+"/"
    analyze_segments(sim_files,geometry, save_to)

In [ ]:
%%time
parent_dir = "trjs_analysis_segments"
simulation_type = 'all' # bug or all
geometry = 'cylindrical'

analysis_delayed = []   
for group in all_groups: # run over all simulations in all the groups
    analysis = delayed(whole_all_group_analyze)(group, parent_dir, simulation_type, geometry)
    analysis_delayed.append(analysis)

In [ ]:
%%time
results = compute(analysis_delayed)